In [1]:
import pandas as pd
import random as rn
import numpy as np
import re
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble

In [2]:
totDF = pd.read_csv('../data/raw/Cleaned_data_set.csv')

In [60]:
def cleanDF (df):
    r1 = re.compile('.*reporting')
    r2 = re.compile('.*imputed')

    cols_to_drop1 = list(filter((r1.match), df.columns))
    cols_to_drop2 = list(filter((r2.match), df.columns))
    cols_to_drop3 = ['admit_NICU']
    cols_to_drop = cols_to_drop1 + cols_to_drop2 + cols_to_drop3

    cols_to_keep = [col for col in df.columns if col not in cols_to_drop]

    X_and_target = df[cols_to_keep + ['admit_NICU']].copy()

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    catDF = X_and_target.select_dtypes(include=object).copy()
    numDF = X_and_target.select_dtypes(include=numerics).copy() #only numeric columns

    le = LabelEncoder()
    catDF = catDF.apply(le.fit_transform)

    concat_df = pd.concat([numDF,catDF],axis=1)
    return concat_df

In [71]:
cl_df = cleanDF(totDF)
nicu_allY = cl_df.loc[cl_df['admit_NICU']==1]
nicu_allN = cl_df.loc[cl_df['admit_NICU']==0]

In [73]:
sample = 5000
sampN = nicu_allN.sample(sample)
sampY = nicu_allY.sample(sample)
samp = pd.concat([sampN,sampY],axis=0)

In [74]:
samp

,birth_year,birth_month,birth_time,birth_day_of_wk,birth_place,mothers_age,mothers_nativity,mothers_residence_status,mothers_race,mothers_hispanic_origin,...,limb_reduc_defect,cleft_lip_or_palate,cleft_palate_only,down_syndr,suspect_chromo_disorder,hypospadias,infant_transferred,infant_living_at_report,infant_breastfed_at_discharge,admit_NICU
261129,2015,6,1129,6,1,21,1,2,1,9,...,0,0,0,1,1,0,0,2,2,0
649711,2017,6,1528,6,1,23,1,1,1,0,...,0,0,0,1,1,0,0,2,0,0
801426,2018,4,1232,7,1,31,1,1,2,0,...,0,0,0,1,1,0,0,2,2,0
135925,2014,2,944,6,1,23,1,1,1,9,...,0,0,0,1,1,0,0,2,2,0
561904,2016,6,1800,6,1,30,1,1,1,0,...,0,0,0,1,1,0,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295980,2015,2,942,1,1,18,1,1,1,9,...,0,0,0,1,1,0,0,2,2,1
110763,2014,3,9999,3,9,26,2,1,99,9,...,1,1,1,3,3,1,1,1,1,1
126536,2014,2,9999,2,9,32,2,1,99,9,...,1,1,1,3,3,1,1,1,1,1
197912,2014,11,9999,3,9,38,2,1,99,9,...,1,1,1,3,3,1,1,1,1,1


In [75]:
samp_target = samp.admit_NICU
samp_X = samp.drop('admit_NICU',axis=1)

In [54]:
sample_per_year = 1000
bal_dwnSmlpY = nicu_allY.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
bal_dwnamlpN = nicu_allN.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
bal_dwnSmlp = pd.concat([bal_dwnSmlpY,bal_dwnamlpN],axis=0)
bal_dwnSmlp.admit_NICU.count()

10000

In [76]:
randomForest = ensemble.RandomForestClassifier()
grid_para_forest = [{
    'n_estimators': np.linspace(50,int(np.sqrt(len(cl_df))),10,dtype=int),
   # 'n_estimators': range(1000,10000,1000),
    #'min_samples_split' : [100,10,2],
    'min_samples_leaf' : range(100,1000,100)
}]
randomForest.set_params(random_state=108)
grid_search_forest = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', 
                                  cv=5, n_jobs=-1)
%time grid_search_forest.fit(samp_X, samp_target)

Wall time: 2min 13s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=108,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'

In [78]:
print(
    '''Random Forest
sample size: {0}
best param : {1}
best score : {2}
r2         : {3}'''\
      .format(len(encoded_target), \
              grid_search_forest.best_params_,\
              grid_search_forest.best_score_, \
              r2_score(samp_target, grid_search_forest.predict(samp_X)))
     )

Random Forest
sample size: 5000
best param : {'min_samples_leaf': 100, 'n_estimators': 366}
best score : 1.0
r2         : 1.0


In [ ]:
y_pred = bestRF.predict(cl_df)
#Finally, use sklearn's confusion_matrix on real and predicted y

from sklearn.metrics import confusion_matrix
print(confusion_matrix(encoded_target, y_pred))

In [6]:
print(
    '''Random Forest
sample size: {0}
best param : {1}
best score : {2}
r2         : {3}'''\
      .format(len(encoded_target), \
              grid_search_forest.best_params_,\
              grid_search_forest.best_score_, \
              r2_score(encoded_target, grid_search_forest.predict(cl_df)))
     )

Random Forest
sample size: 100000
best param : {'min_samples_leaf': 100, 'n_estimators': 109}
best score : 0.94302
r2         : 0.33316310727927356


In [17]:
y_pred = bestRF.predict(cl_df)
#Finally, use sklearn's confusion_matrix on real and predicted y

from sklearn.metrics import confusion_matrix
print(confusion_matrix(encoded_target, y_pred))

[[4510    0    2]
 [  60    0    0]
 [ 408    0   20]]


In [16]:
bestRF = ensemble.RandomForestClassifier()
best_params = grid_search_forest.best_params_
bestRF.set_params(random_state=108,n_jobs= -1,oob_score = True,**best_params)
bestRF.fit(cl_df,encoded_target)

C:\Users\conma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=100, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=109,
                       n_jobs=-1, oob_score=True, random_state=108, verbose=0,
                       warm_start=False)

In [ ]:
sorted( list( zip(feature_cols, bestRF.feature_importances_) ), key = lambda x: x[1], reverse=True )

In [11]:
featureDF = pd.DataFrame({'ScorebestRF' : bestRF.feature_importances_, 'Features' : cl_df.columns})
#featureDF
print(featureDF.sort_values('ScoreBestRF').head(10))

,ScorebestRF,Features
0,0.000429,birth_year
1,0.000683,birth_month
2,0.001998,birth_time
3,0.000535,birth_day_of_wk
4,0.009370,birth_place
...,...,...
98,0.013351,suspect_chromo_disorder
99,0.003977,hypospadias
100,0.019625,infant_transferred
101,0.000732,infant_living_at_report


In [ ]:
print(
'''Random Forest
OOB score: {0}
Feature Importances: {1}
Score: {2}'''\
        .format(bestRF.oob_score_,
                bestRF.feature_importances_,
                bestRF.score))